In [1]:
import pygame, sys
from pygame.locals import *
import os
import numpy
import matplotlib

import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram, plot_state_qsphere

matplotlib.use("Agg")

import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg

pygame.init()
level1 = pygame.image.load("../Images/level1.jpg") 
level1_bg = pygame.transform.scale(level1, (800, 600))
style = {'scale': 0.5}

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def update_circuit(circuit, gate_name, qubit_no):
        
    if gate_name == 'x':
        circuit.x(qubit_no)
    
    if gate_name == 'y':
        circuit.y(qubit_no)
    
    if gate_name == 'z':
        circuit.z(qubit_no)
        
    if gate_name == 'h':
        circuit.h(qubit_no)
        
    if gate_name == 'measure':
        circuit.barrier()
        circuit.measure(0,0)
        circuit.measure(1,1)
        
    if gate_name == 'cnot':
        circuit.cx(qubit_no, (1-qubit_no))
    
    circuit.draw(output='mpl',style=style,filename='test.png')
    circ_img = pygame.image.load('test.png')

    return circuit, circ_img

In [3]:
def measurement(circuit):
    #does measurement of circ image
    #saves pictures of Q sphere and Histogram to variables for next stage

    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=1000)
    counts = job.result().get_counts(circuit)
    plot_histogram(counts).savefig('histogram.png')
    
    backend2 = qiskit.Aer.get_backend('statevector_simulator')
    final_state = execute(circuit,backend2).result().get_statevector()
    plot_state_qsphere(final_state).savefig('qsphere.png')

In [4]:
def level1_circ(x, y, xChange, yChange):
    width = 200
    height = 150
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc = QuantumCircuit(qr, cr)                 
    %matplotlib inline
    qc.draw(output='mpl',style=style,filename='usercircuit.png')
    circ_img = pygame.image.load('usercircuit.png')
    #circ_img = pygame.transform.scale(circ_img, (width, height))
    
    running = True
    commentIter = 0
    #duckHitbox = pygame.Rect(0,0,64,64)
    jump = False
    fall = False
    leftActive = False
    rightActive = False
    upActive = False
    downActive = False
    inst = True
    count = 0
    playerX = 200
    playerY = 200
    xChange = 0
    yChange = 0
    y = 450
    x = 100
    duck = sprite(x, y)
    bg_movement = 0
    qubit_no = 1
    
    #button creation
    '''toggle_img = pygame.image.load('../Images/button.png')
    toggle = pygame.transform.scale(toggle_img, (200, 200))
    toggle_btn = Rect = pygame.Rect(150, 467, 50, 50)
    Icon(toggle, 113, 467)
    
    xgate_img = pygame.image.load('../Images/xgate.png')
    xgate_btn = pygame.transform.scale(xgate_img, (200, 200))
    xgateRect = xgate_btn.get_rect()
    btnX = pygame.Rect(254, 467, 20, 20)
    
    ygate_img = pygame.image.load('../Images/ygate.png')
    ygate_btn = pygame.transform.scale(ygate_img, (200, 200))
    ygateRect = ygate_btn.get_rect()
    btnY = pygame.Rect(394, 467, 20, 20)
    
    zgate_img = pygame.image.load('../Images/zgate.png')
    zgate_btn = pygame.transform.scale(zgate_img, (200, 200))
    zgateRect = zgate_btn.get_rect()
    btnZ = pygame.Rect(535, 467, 20, 20)
    
    hgate_img = pygame.image.load('../Images/hgate.png')
    hgate_btn = pygame.transform.scale(hgate_img, (200, 200))
    hgateRect = hgate_btn.get_rect()
    btnH = pygame.Rect(603, 467, 20, 20)
    
    measure_img = pygame.image.load('../Images/measure.png')
    measure_btn = pygame.transform.scale(measure_img, (200, 200))
    measureRect = measure_btn.get_rect()
    btnM = pygame.Rect(753, 467, 20, 20)
    
    cnot_img = pygame.image.load('../Images/cnot.png')
    cnot_btn = pygame.transform.scale(cnot_img, (200, 200))
    cnotRect = cnot_btn.get_rect()
    btnC = pygame.Rect(780, 467, 20, 20)'''
    
    a = False
    b = False
    c = False
    d = False
    e = False
    f = False
    
    while running:
        screen.fill((0,0,0))
        screen.blit(level1, (0,0))
        
        # Setup
        mx, my = pygame.mouse.get_pos()
        screen.blit(circ_img, (100,50))
        
        click = False
        
        #Instructions
        drQwackDialogue = ["This circuit is empty, let's add some gates!",
                           "To add an X gate, press the X key",
                           "To add a Y gate, press Y",
                           "To add a Z gate, press Z",
                           "To add a Hadamard gate, press H",
                           "To add a CNOT gate, press C",
                           "To toggle which qubit gets a gate, press T",
                           "To perform a measurement, press M",
                           "To see the outcome of the measurement, press N"]
        
        textScreen = pygame.Rect(45, 183, 743, 75)
        duck.comment = drQwackDialogue[commentIter]
        
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                    
                # movement
                if event.key == pygame.K_LEFT:
                    xChange = -2.5
                    leftActive = True
                if event.key == pygame.K_RIGHT:
                    xChange = 2.5
                    rightActive = True
                if event.key == pygame.K_UP:
                    yChange = -2.5
                    upActive = True
                if event.key == pygame.K_DOWN:
                    yChange = 2.5
                    downActive = True
                    
                if event.key == pygame.K_SPACE and commentIter < len(drQwackDialogue)+1:
                    commentIter += 1
                elif event.key == pygame.K_SPACE and commentIter == len(drQwackDialogue)+2:
                    inst = False
                    commentIter = 0
                
                #if duck hits button, gate added to circuit
                if event.key == pygame.K_t:
                    qubit_no = (1-qubit_no)
                
                if event.key == pygame.K_x:
                    a = True
                    
                if event.key == pygame.K_y:
                    b = True
                    
                if event.key == pygame.K_z:
                    c = True
                    
                if event.key == pygame.K_h:
                    d = True
                    
                if event.key == pygame.K_m:
                    e = True
                    
                if event.key == pygame.K_c:
                    f = True
                    
                elif event.key == pygame.K_n:
                    level1_meas(x, y, xChange, yChange)
                    
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    xChange = 0
                    rightActive = False
                    leftActive = False
                if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    yChange = 0
                    upActive = False
                    downActive = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    print(pygame.mouse.get_pos())
        
        #section needed so gates aren't infinitely added           
        if a is True:
            qc, circ_img = update_circuit(qc, 'x', qubit_no)
            a = False
        if b is True:
            qc, circ_img = update_circuit(qc, 'y', qubit_no)
            b = False
        if c is True:
            qc, circ_img = update_circuit(qc, 'z', qubit_no)
            c = False
        if d is True:
            qc, circ_img = update_circuit(qc, 'h', qubit_no)
            d = False
        if e is True:
            qc, circ_img = update_circuit(qc, 'measure', qubit_no)
            e = False
            measurement(qc)
        if f is True:
            qc, circ_img = update_circuit(qc, 'cnot', qubit_no)
            f = False
            
        if commentIter >= len(drQwackDialogue):
            commentIter = 0
        else:
            duck.hasDialogue = True
            duck.comment = drQwackDialogue[commentIter] 
            
        #fix movement
        x += xChange
        if x <= 0:
            x = 0
        elif x >= 800-duck.rect.width:
            x = 800-duck.rect.width

        y += yChange
        if y <= 0:
            y = 0
        elif y >= 600-duck.rect.height:
            y = 600-duck.rect.height
                
        if rightActive:
            duck.RightPressed()
        elif leftActive:
            duck.LeftPressed()
        elif upActive:
            duck.UpPressed()
        elif downActive:
            duck.DownPressed()
        duck.update(x, y, screen)
        
        #Display
        count += 1
        duck.draw(screen)
        #duckHitbox = Icon(icon, x, y)
        pygame.display.update()
        mainClock.tick(120)

In [5]:
'''TODO:
1. Change OG circuit size -> specify width and height of qc.draw()
2. Fix toggle button
3. Keep circuit from getting too big -> counter variable
4. Change location and text of buttons
5. Perform the measurements
'''

'TODO:\n1. Change OG circuit size -> specify width and height of qc.draw()\n2. Fix toggle button\n3. Keep circuit from getting too big -> counter variable\n4. Change location and text of buttons\n5. Perform the measurements\n'